In [1]:
import pandas as pd
import coin2086
import requests

# Load your trades #
The first step in using `coin2086` is to load your trades.  
For more information about the DataFrame format expected by `coin2086`, [see the documentation](https://coin2086.readthedocs.io/en/latest/guide/input.html)

In [2]:
# You may load your trades from a .csv file
trades = pd.read_csv('trades.csv', index_col=0)

# Or build the DataFrame yourself
columns = ['datetime', 'trade_side', 'cryptocurrency', 'quantity', 'price', 'base_currency', 'amount', 'fee']
data = [
 ['2019-10-19 11:10:00', 'BUY', 'BTC', 1.0, 7149.38, 'EUR', 7149.38, 35.7469],
 ['2019-11-14 19:50:00', 'SELL', 'BTC', 0.5, 7844.88, 'EUR', 3922.44, 19.6122],
 ['2020-07-28 10:20:00', 'BUY', 'BTC', 2.0, 9262.42, 'EUR', 18524.84, 92.6242],
 ['2020-09-01 12:20:00', 'BUY', 'ETH', 5.0, 393.58, 'EUR', 1967.9, 9.8395],
 ['2020-09-05 16:50:00', 'SELL', 'BTC', 1.0, 8722.7, 'EUR', 8722.7, 43.6135],
 ['2020-09-08 12:40:00', 'SELL', 'ETH', 5.0, 285.07, 'EUR', 1425.35, 7.12675],
 ['2020-12-20 9:10:00', 'SELL', 'BTC', 0.25, 19223.9, 'EUR', 4805.975, 24.029875],
 ['2021-03-13 23:40:00', 'SELL', 'BTC', 0.25, 50025.17, 'EUR', 12506.2925, 62.5314625]]
trades = pd.DataFrame(data, columns=columns)

In [3]:
trades

,datetime,trade_side,cryptocurrency,quantity,price,base_currency,amount,fee
0,2019-10-19 11:10:00,BUY,BTC,1.00,7149.38,EUR,7149.3800,35.746900
1,2019-11-14 19:50:00,SELL,BTC,0.50,7844.88,EUR,3922.4400,19.612200
2,2020-07-28 10:20:00,BUY,BTC,2.00,9262.42,EUR,18524.8400,92.624200
3,2020-09-01 12:20:00,BUY,ETH,5.00,393.58,EUR,1967.9000,9.839500
4,2020-09-05 16:50:00,SELL,BTC,1.00,8722.70,EUR,8722.7000,43.613500
5,2020-09-08 12:40:00,SELL,ETH,5.00,285.07,EUR,1425.3500,7.126750
6,2020-12-20 9:10:00,SELL,BTC,0.25,19223.90,EUR,4805.9750,24.029875
7,2021-03-13 23:40:00,SELL,BTC,0.25,50025.17,EUR,12506.2925,62.531463


# Call `coin2086.compute_taxable_pnl()` #

## Without initial portfolio ##

In [4]:
year = 2020
allowance = 305

In [5]:
form2086, taxable_profit = coin2086.compute_taxable_pnls(trades, year=year)
print(f"Total taxable profit for year {year}: {taxable_profit:.2f} euros")
if taxable_profit > allowance:
    expected_tax = (taxable_profit - allowance) * 0.3
    print(f"Expected tax amount: {expected_tax:.2f} euros")
    print(f"Fill form 2086 with the information below" )
else:
    print(f"Your profit is below {allowance} euros, your do not have to report")

Valuate done
Total taxable profit for year 2020: 2038.50 euros
Expected tax amount: 520.05 euros
Fill form 2086 with the information below


In [6]:
form2086

,Description,Date de la cession [datetime],Valeur globale portefeuille [portfolio_value],Prix de cession [amount],Frais de cession [fee],Prix de cession net des frais [amount_net],Prix total d'acquistion [portfolio_purchase_price],Fractions de capital initial [purchase_price_fraction_sum],Prix total d'acquistion net [portfolio_purchase_price_net],Plus-values et moins-values [pnl]
4,SELL 1.00 BTC,2020-09-05 16:50:00,23310.95,8722.700,43.613500,8679.086500,27780.3306,3592.563450,24187.767150,-371.708792
5,SELL 5.00 ETH,2020-09-08 12:40:00,14189.21,1425.350,7.126750,1418.223250,27780.3306,12643.358742,15136.971858,-102.332358
6,SELL 0.25 BTC,2020-12-20 09:10:00,28835.85,4805.975,24.029875,4781.945125,27780.3306,14163.914350,13616.416250,2512.542417


## With initial portfolio ## 
If your trades DataFrame does not include **all** your trades since you bought your first cryptocurrency, you will have to pass an initial portfolio  
[See the documentation](https://coin2086.readthedocs.io/en/latest/guide/input.html#initial-portfolio) for more information

In [7]:
# You bought 1 BTC for 2400 euros in 2017 and 1 ETH for 290 euros
# but you have lost the trades
initial_portfolio = {'BTC': 1.0, 'ETH': 1.0}
initial_purchase_price = 2400 + 290
form2086, taxable_profit = coin2086.compute_taxable_pnls(trades, 2020, 
                                                         initial_portfolio=initial_portfolio,
                                                         initial_purchase_price=initial_purchase_price)
print(f"Total taxable profit for year {year}: {taxable_profit:.2f} euros")
if taxable_profit > allowance:
    expected_tax = (taxable_profit - allowance) * 0.3
    print(f"Expected tax amount: {expected_tax:.2f} euros")
    print(f"Fill form 2086 with the information below" )
else:
    print(f"Your profit is below {allowance} euros, your do not have to report")

Valuate done
Total taxable profit for year 2020: 4150.51 euros
Expected tax amount: 1153.65 euros
Fill form 2086 with the information below


In [8]:
form2086

,Description,Date de la cession [datetime],Valeur globale portefeuille [portfolio_value],Prix de cession [amount],Frais de cession [fee],Prix de cession net des frais [amount_net],Prix total d'acquistion [portfolio_purchase_price],Fractions de capital initial [purchase_price_fraction_sum],Prix total d'acquistion net [portfolio_purchase_price_net],Plus-values et moins-values [pnl]
4,SELL 1.00 BTC,2020-09-05 16:50:00,32334.49,8722.700,43.613500,8679.086500,30470.3306,2442.844208,28027.486392,1118.263504
5,SELL 5.00 ETH,2020-09-08 12:40:00,22983.52,1425.350,7.126750,1418.223250,30470.3306,10003.667204,20466.663396,148.959070
6,SELL 0.25 BTC,2020-12-20 09:10:00,48593.34,4805.975,24.029875,4781.945125,30470.3306,11272.931384,19197.399216,2883.285335


**You are done !**

# Additional tips #

If you want to use this Binder to compute your own taxable profit, you may be interested in loading your trades from a Google Sheet.

In [9]:
# Sheet public URL: 
WORBOOK_KEY = "16-L3K1JEg_jfQHZyTnYNoHQK-Dg4Rfx8Ic_jAFgYeQQ" # The WORBOOK_KEY can be found in the public URL your sheet
SHEET_NAME = 'trades'
trades = pd.read_csv(f"https://docs.google.com/spreadsheets/d/{WORBOOK_KEY}/gviz/tq?tqx=out:csv&sheet={SHEET_NAME}")

In [10]:
trades

,datetime,trade_side,cryptocurrency,quantity,price,base_currency,amount,fee
0,2019-10-19 11:10:00,BUY,BTC,1.00,7149.38,EUR,7149.3800,35.746900
1,2019-11-14 19:50:00,SELL,BTC,0.50,7844.88,EUR,3922.4400,19.612200
2,2020-07-28 10:20:00,BUY,BTC,2.00,9262.42,EUR,18524.8400,92.624200
3,2020-09-01 12:20:00,BUY,ETH,5.00,393.58,EUR,1967.9000,9.839500
4,2020-09-05 16:50:00,SELL,BTC,1.00,8722.70,EUR,8722.7000,43.613500
5,2020-09-08 12:40:00,SELL,ETH,5.00,285.07,EUR,1425.3500,7.126750
6,2020-12-20 9:10:00,SELL,BTC,0.25,19223.90,EUR,4805.9750,24.029875
7,2021-03-13 23:40:00,SELL,BTC,0.25,50025.17,EUR,12506.2925,62.531463
